In [1]:
import os

In [2]:
%pwd  

'd:\\Final Year Project\\Capstone-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Final Year Project\\Capstone-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epoch: int  
    params_image_size: list

In [6]:
from src.KidneyProject.constants import *
from src.KidneyProject.utils.common import read_yaml,create_directories
import tensorflow as tf  

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_fiepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_fiepath) 
    
    def get_training_path(self) -> TrainingConfig:
        training = self.config.training    
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"CT Kidney Dataset")
        create_directories([
            Path(training.root_dir)
        ]) 
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epoch=params.EPOCHS,  
            params_image_size=params.IMAGE_SIZE
        )  
        
        return training_config 

In [8]:
import os
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.model.summary()

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255.0,
            validation_split=0.1
        )
        self.callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0.0001,
            patience=5,
            verbose=1,
            mode="auto",
            baseline=None,
            restore_best_weights=False
        )
 
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            image_size=self.config.params_image_size[:-1], 
            validation_split=0.1,
            subset="training",
            seed=123,
            label_mode='categorical'
        )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            image_size=self.config.params_image_size[:-1], 
            validation_split=0.2,
            subset="validation",
            seed=123,
            label_mode='categorical'
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            validation_data=self.valid_generator,
            callbacks=self.callback
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_path()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()  
except Exception as e:
    raise e     

[2024-02-29 09:49:55,515: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 09:49:55,518: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 09:49:55,519: INFO: common: created directory at: artifacts\training]


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     